<a href="https://colab.research.google.com/github/Tavo826/PyTorch/blob/main/IntroductionPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

torch.__version__

'1.4.0'

In [ ]:
x_zero = torch.empty(5,3)
x_rand = torch.rand(5,3)

x_array = torch.tensor([5.5, 3], dtype=torch.int)
x_array

y_array = torch.tensor([6, 4])

x_array + y_array

tensor([11,  7])

In [ ]:
result = torch.empty(5, 3)
torch.add(x_array, y_array, out=result)
print(result)

tensor([11.,  7.])


In [ ]:
y_array.add_(x_array)

x_array[0:2]
x_array[0].item()

5

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
out = z.mean()

print(z, out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [ ]:
print(x.grad)
#Backpropagar desde el tensor original que acumulaba los gradietntes
#hasta la operación a la que se ha llegado
out.backward()

None


In [ ]:
#Variación por cada incremento de una unidad en el primer componente del vector original
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
  y = y * 2

print(y)

tensor([  12.4525, 1074.3367,  579.7873], grad_fn=<MulBackward0>)


RED NEURONAL

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) #10 salidas por clasificación de números (Mnist)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
net.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [ ]:
#Simula una imagen
input = torch.randn(1, 1, 32, 32)
input

tensor([[[[-0.1784,  0.1406,  0.3635,  ...,  0.0735, -0.3627, -1.2926],
          [ 0.1347, -0.3066,  0.3995,  ..., -0.0505, -0.4192,  0.3335],
          [ 0.0170,  0.1443,  0.8315,  ...,  2.3371, -0.4406,  0.8918],
          ...,
          [-0.4224, -0.7987,  0.7079,  ..., -0.5570,  0.3124,  0.3444],
          [-0.5268,  0.2644, -1.8729,  ..., -0.0998, -2.0286, -0.6695],
          [-1.5194,  1.8930,  0.2054,  ...,  0.3770, -1.1775,  0.9173]]]])

In [ ]:
out = net(input)
print(out)

tensor([[ 0.0667,  0.0255,  0.0488, -0.0425,  0.1033, -0.0710,  0.0501,  0.0173,
         -0.0549,  0.1001]], grad_fn=<AddmmBackward>)


Función de pérdida

In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7453, grad_fn=<MseLossBackward>)


In [ ]:
#Limpia los gradientes anteriores
# net.zero_grad()

loss.backward(retain_graph=True)

In [ ]:
#Gradiente capa 1
print(net.conv1.bias.grad)

tensor([ 0.0037,  0.0033, -0.0138,  0.0079, -0.0098,  0.0002])


In [ ]:
Optimización

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update